<a href="https://colab.research.google.com/github/phanmanhtung/Natural-Language-Processing-Course/blob/master/Programming_Assignment_1_Bigram_Language_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
Programming Assignment 1: Bigram Language Models

Full name: PHAN MANH TUNG
Student number: USTHBI8-160
Email address: tungpm.b8160@st.usth.edu.vn

In [0]:
# Data
!rm -f wiki-en-train.word
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word
    
!rm -f wiki-en-test.word
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word

!rm -f 02-train-input.txt
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/test/02-train-input.txt

!rm -f 02-train-answer.txt
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/test/02-train-answer.txt

--2020-02-08 14:44:48--  https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203886 (199K) [text/plain]
Saving to: ‘wiki-en-train.word’

wiki-en-train.word  100%[===================>] 199.11K  --.-KB/s    in 0.05s   

2020-02-08 14:44:48 (3.99 MB/s) - ‘wiki-en-train.word’ saved [203886/203886]

--2020-02-08 14:44:51--  https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26989 (26K) [t

In [0]:
# Part 1: Estimating probabilities
# Part 1.0: Function train_unigram()

from collections import defaultdict

# UNIGRAM
def train_unigram(train_file, model_file):
    counts = defaultdict(int)  # count the n-gram (a map count)
    total_count = 0
    #context_counts = defaultdict(int)   # count the context
    with open(train_file) as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')

            for word in words:
              counts[word] += 1
              total_count +=1
              pass # do nothing

    # Save probabilities to the model file            
    with open(model_file, 'w') as fo:
        for ngram, count in counts.items():

            fo.write((ngram + "\t" + str(counts[ngram]/total_count))+ "\n")

            pass

In [0]:
train_unigram('02-train-input.txt', 'train-answer.txt')

In [0]:
!cat train-answer.txt

a	0.25
b	0.25
c	0.125
</s>	0.25
d	0.125


In [0]:
# Part 1.1: Function train_bigram()

from collections import defaultdict

# BIGRAM
def train_bigram(train_file, model_file):
    counts = defaultdict(int)  # count the n-gram (a map count)
    total_count = 0
    context_counts = defaultdict(int)
    #context_counts = defaultdict(int)   # count the context
    with open(train_file) as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')

            for i in range(1, len(words)):
                # Add bigram and bigram context
                counts[words[i-1]+ " "+ words[i]] += 1             
                context_counts[words[i-1]] += 1

                # Add unigram and unigram context
                total_count += 1
                counts[words[i]] += 1
                context_counts[""] += 1
                pass # do nothing

    # Save probabilities to the model file            
    with open(model_file, 'w') as fo:
        for ngram, count in counts.items():
            # split ngram into an array of words
            ngram_split = ngram.split()

            # BIGRAM
            if len(ngram_split) > 1: # 
              fo.write((ngram + "\t" + str(counts[ngram]/context_counts[ngram_split[0]]))+"\n")

            # UNIGRAM
            else:
              fo.write((ngram + "\t" + str(counts[ngram]/total_count))+"\n")

            pass

In [0]:
train_bigram('02-train-input.txt', 'train-answer.txt')

In [0]:
!cat train-answer.txt

<s> a	1.0
a	0.25
a b	1.0
b	0.25
b c	0.5
c	0.125
c </s>	1.0
</s>	0.25
b d	0.5
d	0.125
d </s>	1.0


In [0]:
# Part 1.2: load the model file

def load_bigram_model(model_file):
    """Load the model file

    Args:
        model_file (str): Path to the model file
    
    Returns:
        probs (dict): Dictionary object that map from ngrams to their probabilities
    """
    probs = {}
    with open(model_file, 'r') as f:
        for line in f:
            element = line.split("\t")
            probs[element[0]] = float(element[1])
            pass
    return probs

In [0]:
probs = load_bigram_model('train-answer.txt')
probs

{'</s>': 0.25,
 '<s> a': 1.0,
 'a': 0.25,
 'a b': 1.0,
 'b': 0.25,
 'b c': 0.5,
 'b d': 0.5,
 'c': 0.125,
 'c </s>': 1.0,
 'd': 0.125,
 'd </s>': 1.0}

In [0]:
print(probs["a b"])
print(probs.get("a b"))

1.0
1.0


In [0]:
# Part 2: Evaluating Bigram Language Model

import math


def test_bigram(test_file, model_file, lambda2=0.95, lambda1=0.95, N=1000000):
    W = 0 # Total word count
    H = 0
    probs = load_bigram_model(model_file)
    with open(test_file, 'r') as f:
        for line in f:
            # get rid of the whitespace
            line = line.strip()
            if line == '':
                continue

            # split line into array of words
            words = line.split()

            # add to the end and beginning
            words.append('</s>')
            words.insert(0, '<s>')
            
            for i in range(1, len(words)):  # Note: starting at 1, after <s>
                # TODO: Write code to calculate smoothed unigram probabilties
                # and smoothed bigram probabilities
                # You should use calculate p1 as smoothed unigram probability
                # and p2 as smoothed bigram probability

                # If the unigram exists in the dictionary
                if (probs.get(words[i])):
                  p1 = lambda1 * probs.get(words[i]) + (1-lambda1) * (1/N)
                else: # We assume the value of probs.get(words[i]) to be 0
                  p1 = lambda1 * 0 + (1-lambda1) * (1/N) 
                
                # If the bigram exists in the dictionary
                if (probs.get(words[i-1] + " " + words[i])):
                  p2 = lambda2 * probs.get(words[i-1] + " " + words[i])+ (1-lambda2) * p1
                else: # We assume the value of probs.get(words[i-1] + " " + words[i]) to be 0
                  p2 = lambda2 * 0 + (1-lambda2) * p1

                W += 1  # Count the words
                H += -math.log2(p2) # We use logarithm to avoid underflow
    H = H/W
    P = 2**H
    
    print("Entropy: {}".format(H))
    print("Perplexity: {}".format(P))

    return P


# So the error occur when there is no "In computational" in the testing set, which appear None type in the probs dictionary
# The solution is to quantify the missing value (0)

In [0]:
train_bigram('wiki-en-train.word', 'bigram_model.txt')
test_bigram('wiki-en-test.word', 'bigram_model.txt')

Entropy: 11.284859117885485
Perplexity: 2495.0605603552463


2495.0605603552463